<a href="https://colab.research.google.com/github/alexjosesilva/implementacao-de-rede-neural-adaptado/blob/main/Projeto_final_Implementa%C3%A7%C3%A3o_de_Rede_Neural_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Projeto Final - Implementação de Rede Neural**

Equipe:
1.   Alex José Silva


Gera:
 - métricas no console
 - comparação com baseline do exemplo original
 - gráfico da fronteira de decisão (para datasets 2D)
 - arquivo results.json com os números



In [ ]:
import argparse
import json
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam

In [ ]:
ACTIVATIONS = {"relu": "relu", "sigmoid": "sigmoid", "tanh": "tanh", "elu": "elu", "selu": "selu", "linear": "linear"}

In [ ]:
def build_model(input_dim: int, layers: list[int], activations: list[str], output_activation: str = "sigmoid", lr: float = 0.001, optimizer: str = "adam"):
    model = Sequential()
    if not layers:
        raise ValueError("Forneça ao menos uma camada oculta em --layers")
    if len(activations) == 1 and len(layers) > 1:
        activations = activations * len(layers)
    if len(activations) != len(layers):
        raise ValueError("Número de funções de ativação deve bater com número de camadas ocultas")

    # Primeira camada com input_dim
    model.add(Dense(layers[0], input_dim=input_dim, activation=ACTIVATIONS.get(activations[0], activations[0])))
    # Demais camadas
    for units, act in zip(layers[1:], activations[1:]):
        model.add(Dense(units, activation=ACTIVATIONS.get(act, act)))
    # Saída binária
    model.add(Dense(1, activation=output_activation))

    if optimizer.lower() == "sgd":
        opt = SGD(learning_rate=lr)
    else:
        opt = Adam(learning_rate=lr)
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])  # melhor que MSE p/ classificação
    return model

In [ ]:
def load_dataset(name: str, n_samples: int, noise: float, random_state: int = 42):
    name = name.lower()
    if name == "moons":
        X, y = datasets.make_moons(n_samples=n_samples, noise=noise, random_state=random_state)
        two_d = True
    elif name == "circles":
        X, y = datasets.make_circles(n_samples=n_samples, noise=noise, factor=0.5, random_state=random_state)
        two_d = True
    elif name == "iris":
        iris = datasets.load_iris()
        # binariza: classe 0 vs não-0 só para manter saída sigmoide
        y = (iris.target != 0).astype(int)
        # usa duas features p/ plot 2D
        X = iris.data[:, [2, 3]]  # petal length, petal width
        two_d = True
    else:
        raise ValueError("Dataset inválido. Use: moons | circles | iris")
    return X.astype(np.float32), y.astype(int), two_d

In [ ]:
# This cell is no longer needed as plot_decision_boundary is moved to the main cell.

In [ ]:
def run_baseline(X, y, lr: float, epochs: int, batch_size: int, random_state: int = 42):
    # Replica (aprox.) a cabeça do exemplo4.py: Dense(5,relu)->Dense(5,tanh)->Dense(1,sigmoid), SGD lr=0.1, MSE
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.3, stratify=y, random_state=random_state)

    baseline = Sequential()
    baseline.add(Dense(5, input_dim=X.shape[1], activation='relu'))
    baseline.add(Dense(5, activation='tanh'))
    baseline.add(Dense(1, activation='sigmoid'))

    opt = SGD(learning_rate=lr)
    baseline.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
    baseline.fit(X_train, y_train, epochs=epochs, verbose=False, batch_size=batch_size)
    y_pred = (baseline.predict(X_test, verbose=0).ravel() > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return acc, cm

In [ ]:
import argparse
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam


def plot_decision_boundary(model, X, y, title: str, outpath: Path):
    # Apenas para dados 2D
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300), np.linspace(y_min, y_max, 300))
    grid = np.c_[xx.ravel(), yy.ravel()]
    Z = model.predict(grid, verbose=0)
    Z = (Z.ravel() > 0.5).astype(int)
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(6, 5))
    plt.contourf(xx, yy, Z, alpha=0.3)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolor="k")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(outpath)
    plt.close()

def main():
    p = argparse.ArgumentParser()
    p.add_argument('--dataset', default='moons', choices=['moons', 'circles', 'iris'])
    p.add_argument('--n-samples', type=int, default=300)
    p.add_argument('--noise', type=float, default=0.2)
    p.add_argument('--layers', type=int, nargs='+', default=[8, 8])
    p.add_argument('--activations', nargs='+', default=['relu', 'relu'])
    p.add_argument('--epochs', type=int, default=200)
    p.add_argument('--batch-size', type=int, default=16)
    p.add_argument('--lr', type=float, default=0.01)
    p.add_argument('--optimizer', default='adam', choices=['adam', 'sgd'])
    p.add_argument('--cv', type=int, default=0, help='Número de folds para cross-validation (0 = holdout)')
    p.add_argument('--seed', type=int, default=42)
    # Use parse_known_args() to ignore the arguments passed by Colab
    args, unknown = p.parse_known_args()


    outdir = Path('outputs')
    outdir.mkdir(exist_ok=True)

    X, y, two_d = load_dataset(args.dataset, args.n_samples, args.noise, args.seed)

    # Escalonamento
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)

    results = {"dataset": args.dataset, "n_samples": int(len(X)), "noise": float(args.noise)}

    # ------------------------------
    # Baseline (do exemplo original)
    # ------------------------------
    base_acc, base_cm = run_baseline(X, y, lr=0.1, epochs=100, batch_size=5)  # fiel ao exemplo
    results["baseline_acc"] = float(base_acc)
    results["baseline_cm"] = base_cm.tolist()

    # ------------------------------
    # Modelo customizado
    # ------------------------------
    if args.cv and args.cv > 1:
        skf = StratifiedKFold(n_splits=args.cv, shuffle=True, random_state=args.seed)
        accs = []
        for fold, (tr, te) in enumerate(skf.split(Xs, y), start=1):
            model = build_model(Xs.shape[1], args.layers, args.activations, lr=args.lr, optimizer=args.optimizer)
            model.fit(Xs[tr], y[tr], epochs=args.epochs, verbose=False, batch_size=args.batch_size)
            yhat = (model.predict(Xs[te], verbose=0).ravel() > 0.5).astype(int)
            accs.append(accuracy_score(y[te], yhat))
        acc = float(np.mean(accs))
        results["custom_acc_cv_mean"] = acc
        print(f"Acurácia média (CV {args.cv}): {acc:.4f}")
    else:
        X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.3, stratify=y, random_state=args.seed)
        model = build_model(Xs.shape[1], args.layers, args.activations, lr=args.lr, optimizer=args.optimizer)
        model.fit(X_train, y_train, epochs=args.epochs, verbose=False, batch_size=args.batch_size)
        yhat = (model.predict(X_test, verbose=0).ravel() > 0.5).astype(int)
        acc = accuracy_score(y_test, yhat)
        cm = confusion_matrix(y_test, yhat)
        print(classification_report(y_test, yhat, digits=4))
        results["custom_acc"] = float(acc)
        results["custom_cm"] = cm.tolist()
        print(f"Acurácia (custom): {acc:.4f}")

        # Fronteira de decisão (quando possível)
        if two_d:
            grid_title = f"{args.dataset} | camadas {args.layers} | acts {args.activations}"
            plot_decision_boundary(model, scaler.transform(X), y, grid_title, outdir / f"decision_{args.dataset}.png")

    # Comparação
    if "custom_acc" in results:
        delta = results["custom_acc"] - results["baseline_acc"]
        print(f"\nComparação vs baseline: Δacc = {delta:+.4f}")
        results["delta_acc_vs_baseline"] = float(delta)
    elif "custom_acc_cv_mean" in results:
        delta = results["custom_acc_cv_mean"] - results["baseline_acc"]
        print(f"\nComparação vs baseline (CV): Δacc = {delta:+.4f}")
        results["delta_acc_vs_baseline"] = float(delta)

    # Persistir resultados
    with open(outdir / 'results.json', 'w') as f:
        json.dump(results, f, indent=2)
    print("Resultados salvos em outputs/results.json")

if __name__ == '__main__':
    main()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


              precision    recall  f1-score   support

           0     1.0000    0.9111    0.9535        45
           1     0.9184    1.0000    0.9574        45

    accuracy                         0.9556        90
   macro avg     0.9592    0.9556    0.9555        90
weighted avg     0.9592    0.9556    0.9555        90

Acurácia (custom): 0.9556

Comparação vs baseline: Δacc = +0.0556
Resultados salvos em outputs/results.json


In [ ]:
# The main function is now called within the __name__ == '__main__' block in the previous cell.
# This cell is no longer needed.